# Data Processing

The purpose of this notebook is to download data and process it. As a result we will get a dataset ready for further analyses and modeling.

# Table of Contents

[1. Imports & Environment Configuration](#Imports-&-Environment-Configuration)

# Imports & Environment Configuration

In [1]:
%load_ext autoreload

import glob
import json
import matplotlib.pyplot as plt
import os
import pandas as pd

if os.path.basename(os.getcwd()) == 'notebooks':
    # Make it possible to import modules
    import sys
    sys.path.append("../") 

from src.data.data_loading.bike_station_locations import BikeStationsLocations
from src.data.data_loading.blob_downloader import BlobDownloader

In [2]:
# let matplotlib plots be part of Jupyter Notebook
%matplotlib inline

# set up higher resolution for matplotlib plots
%config InlineBackend.figure_format = 'retina'

## Environment Configuration Variables

In [3]:
# Determine the root folder of the project. The current working directory 
# may vary depending on whether the notebook is run in Jupyter Notebook or VS Code
if os.path.basename(os.getcwd()) == 'notebooks':
    project_root_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))   
else:
    project_root_folder = os.getcwd()
    
# Set up paths to data folders
data_raw_folder     = os.path.join(project_root_folder, 'data', 'raw')
data_interim_folder = os.path.join(project_root_folder, 'data', 'interim')

# Download data

Download all needed data and clean it.

## Bike rental records

### Download

In [4]:
# Provide config details for the Azure Storage container with data.
account_name='<storage_account_name>'
account_key='<storage_account_key>'
container_name='<storage_container_name>' 

# Alternatively, they can be grabbed from the 'local.settings.json' file 
# (used by Azure Functions), if it exists.
local_settings_file_path = os.path.join(project_root_folder, 
                                        'src', 'azurefunctions', 'local.settings.json')
if os.path.exists(local_settings_file_path):
    with open(local_settings_file_path, 'r') as f:
        local_settings = json.load(f)
        
    account_name = local_settings['Values']['storage_account_name']
    account_key = local_settings['Values']['storage_account_key']
    container_name = local_settings['Values']['storage_container_name']

In [5]:
# Download all data from Azure Blob Storage and save it locally 
blob_downloader = BlobDownloader(account_name, account_key, container_name)
blob_downloader.download_blobs_and_save(data_raw_folder)

File already downloaded: Historia_przejazdow_2019-10-10_15_37_27.csv
File already downloaded: Historia_przejazdow_2019-10-11_15_34_34.csv
File already downloaded: Historia_przejazdow_2019-10-16_14_42_31.csv
File already downloaded: Historia_przejazdow_2019-10-17_14_45_15.csv
File already downloaded: Historia_przejazdow_2019-10-18_14_45_31.csv
File already downloaded: Historia_przejazdow_2019-10-19_14_44_34.csv
File already downloaded: Historia_przejazdow_2019-10-1_15_28_35.csv
File already downloaded: Historia_przejazdow_2019-10-20_14_46_37.csv
File already downloaded: Historia_przejazdow_2019-10-21_14_49_54.csv
File already downloaded: Historia_przejazdow_2019-10-22_14_49_13.csv
File already downloaded: Historia_przejazdow_2019-10-23_14_48_12.csv
File already downloaded: Historia_przejazdow_2019-10-24_14_54_25.csv
File already downloaded: Historia_przejazdow_2019-10-2_15_32_45.csv
File already downloaded: Historia_przejazdow_2019-10-3_15_31_46.csv
File already downloaded: Historia_prz

### Load multiple csv files

In [6]:
dfs = []

# Get filenames and load data to 
for filename in glob.glob(os.path.join(data_raw_folder, 'Historia_przejazdow_*.csv')):
    dfs.append(pd.read_csv(filename, parse_dates=['Data wynajmu', 'Data zwrotu']))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

In [7]:
print(big_frame.shape)

(5562129, 7)


In [8]:
big_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5562129 entries, 0 to 5562128
Data columns (total 7 columns):
UID wynajmu       int64
Numer roweru      int64
Data wynajmu      datetime64[ns]
Data zwrotu       datetime64[ns]
Stacja wynajmu    object
Stacja zwrotu     object
Czas trwania      object
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 297.0+ MB


### Clean

#### Drop duplicates

In [9]:
# Last record is considered as unique and rest of the same values as duplicate
big_frame.drop_duplicates(subset="UID wynajmu", keep='last', inplace=True)

# Sort it
big_frame.sort_values(by='UID wynajmu', inplace=True)

In [10]:
print(big_frame.shape)

(430531, 7)


In [11]:
big_frame.head()

,UID wynajmu,Numer roweru,Data wynajmu,Data zwrotu,Stacja wynajmu,Stacja zwrotu,Czas trwania
5549242,76160681,57719,2019-06-26 00:00:10,2019-06-26 00:06:19,Skarbowców / Wietrzna,Krzycka / Aleja Karkonoska (Park Południowy),00:06:09
5549243,76160684,650480,2019-06-26 00:00:13,2019-06-26 00:06:59,Rynek,Plac Legionów,00:06:46
5549244,76160686,650988,2019-06-26 00:00:15,2019-06-26 00:13:32,Poza oficjalną stacją,Wałbrzyska - pętla tramwajowa,00:13:17
5549245,76160697,57603,2019-06-26 00:00:21,2019-06-26 00:23:53,Plac Uniwersytecki (UWr),Legnicka / Wejherowska,00:23:32
5549246,76160714,650067,2019-06-26 00:00:40,2019-06-26 00:04:40,Powstańców Śląskich (Arkady Wrocławskie),Powstańców Śląskich (Arkady Wrocławskie),00:04:00


In [12]:
big_frame.tail()

,UID wynajmu,Numer roweru,Data wynajmu,Data zwrotu,Stacja wynajmu,Stacja zwrotu,Czas trwania
3088454,88278295,650284,2019-10-22 23:52:00,2019-10-22 23:52:00,Fosa Miejska,Fosa Miejska,00:00:00
3088455,88278312,650001,2019-10-22 23:53:00,2019-10-22 23:53:00,Drobnera / Dubois,Drobnera / Dubois,00:00:00
3088456,88278321,650171,2019-10-22 23:53:00,2019-10-22 23:57:00,Drobnera / Plac Bema,Żeromskiego / Daszyńskiego,00:04:00
3088457,88278324,57846,2019-10-22 23:53:00,2019-10-22 23:59:00,al. Brücknera / Kwidzyńska,al. Kochanowskiego / Śniadeckich,00:06:00
3088458,88278425,57435,2019-10-22 23:58:00,2019-10-22 23:58:00,Plac Uniwersytecki (UWr),Plac Uniwersytecki (UWr),00:00:00


In [13]:
# rename columns to make it legible for English-speaking audience
big_frame.columns = ['UID', 'Bike number', 'Rental datetime', 'Return datetime', 
                     'Rental station', 'Return station', 'Duration']

In [14]:
big_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430531 entries, 5549242 to 3088458
Data columns (total 7 columns):
UID                430531 non-null int64
Bike number        430531 non-null int64
Rental datetime    430531 non-null datetime64[ns]
Return datetime    430531 non-null datetime64[ns]
Rental station     430531 non-null object
Return station     430531 non-null object
Duration           430531 non-null object
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 26.3+ MB


### Save interim results

In [15]:
# write interim file
bike_rentals_interim_path = os.path.join(data_interim_folder, 'bike_rental_records.csv')
big_frame.to_csv(path_or_buf=bike_rentals_interim_path, index=False)

# Load data

## Bike rentals records

In [16]:
# Load data and set proper data types
bike_rentals_df = pd.read_csv(bike_rentals_interim_path, 
                              parse_dates=['Rental datetime', 'Return datetime'], 
                              index_col='UID')

bike_rentals_df['Duration'] = pd.to_timedelta(bike_rentals_df['Duration'])

In [17]:
# just checking - to make sure data types are as we want
bike_rentals_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430531 entries, 76160681 to 88278425
Data columns (total 6 columns):
Bike number        430531 non-null int64
Rental datetime    430531 non-null datetime64[ns]
Return datetime    430531 non-null datetime64[ns]
Rental station     430531 non-null object
Return station     430531 non-null object
Duration           430531 non-null timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), object(2), timedelta64[ns](1)
memory usage: 23.0+ MB


In [18]:
bike_rentals_df.sample(5)

,Bike number,Rental datetime,Return datetime,Rental station,Return station,Duration
UID,,,,,,
79297198,650273,2019-07-22 18:35:00,2019-07-22 18:54:00,Kamienna / Borowska (Aquapark),Grabiszyńska / Aleja Hallera,00:19:00
83153072,57853,2019-08-26 15:20:00,2019-08-26 15:21:00,Bałtycka / Żmigrodzka,Bałtycka / Żmigrodzka,00:01:00
84284121,57591,2019-09-05 13:02:00,2019-09-05 13:13:00,Zaporoska / Grabiszyńska,Powstańcow Śląskich / Aleja Hallera,00:11:00
84308740,57611,2019-09-05 16:17:00,2019-09-05 16:49:00,Stacyjna (Dworzec Mikołajów),Plac Grunwaldzki / Polaka,00:32:00
79384112,650483,2019-07-23 15:15:00,2019-07-23 15:31:00,Zaporoska / Wielka / Krucza,Klecińska / Duńska,00:16:00


In [19]:
# check if we have any missing data
bike_rentals_df.isna().sum()

Bike number        0
Rental datetime    0
Return datetime    0
Rental station     0
Return station     0
Duration           0
dtype: int64

## Bike station details

In [20]:
bike_stations_df = BikeStationsLocations().load_data()

In [21]:
bike_stations_df

,Bike station,Latitude,Longitude
0,Plac Dominikański (Galeria Dominikańska),51.108004,17.039528
1,"Dworzec Główny, południe",51.097108064432,17.036109566688538
2,Rynek,51.109782,17.030175
3,Dworzec Główny,51.09975,17.036228
4,Nowowiejska / Jedności Narodowej,51.124879,17.045844
...,...,...,...
198,pl. Orląt Lwowskich,51.10823,17.02138
199,Komandorska / Kamienna,51.09009,17.0235
200,"Wrocław Stadion, stacja kolejowa",51.13707,16.94095
201,"Wrocław Leśnica, stacja kolejowa",51.14323,16.86627


In [22]:
# check if we have any missing data
bike_stations_df.isna().sum()

Bike station    0
Latitude        0
Longitude       0
dtype: int64

## Join datasets

In [23]:
bike_rentals_df = pd.merge(bike_rentals_df, 
                           bike_stations_df, 
                           how='left',
                           left_on='Rental station',
                           right_on='Bike station')

In [24]:
# define more meaningful names for coordinates columns
cols = [f'Rental station {str.lower(column)}' 
            if column in ['Latitude', 'Longitude'] else column
                for column in bike_rentals_df.columns]
bike_rentals_df.columns = cols

# drop unnecessary
bike_rentals_df.drop(['Bike station'], axis=1, inplace=True)

In [25]:
bike_rentals_df.head()

,Bike number,Rental datetime,Return datetime,Rental station,Return station,Duration,Rental station latitude,Rental station longitude
0,57719,2019-06-26 00:00:10,2019-06-26 00:06:19,Skarbowców / Wietrzna,Krzycka / Aleja Karkonoska (Park Południowy),00:06:09,51.07329,16.99485
1,650480,2019-06-26 00:00:13,2019-06-26 00:06:59,Rynek,Plac Legionów,00:06:46,51.109782,17.030175
2,650988,2019-06-26 00:00:15,2019-06-26 00:13:32,Poza oficjalną stacją,Wałbrzyska - pętla tramwajowa,00:13:17,NaN,NaN
3,57603,2019-06-26 00:00:21,2019-06-26 00:23:53,Plac Uniwersytecki (UWr),Legnicka / Wejherowska,00:23:32,51.113871,17.034484
4,650067,2019-06-26 00:00:40,2019-06-26 00:04:40,Powstańców Śląskich (Arkady Wrocławskie),Powstańców Śląskich (Arkady Wrocławskie),00:04:00,51.099713,17.027905


In [26]:
bike_rentals_df = pd.merge(bike_rentals_df, 
                           bike_stations_df, 
                           how='left',
                           left_on='Return station',
                           right_on='Bike station')

In [27]:
# define more meaningful names for coordinates columns
cols = [f'Return station {str.lower(column)}' 
            if column in ['Latitude', 'Longitude'] else column
                for column in bike_rentals_df.columns]
bike_rentals_df.columns = cols

# drop unnecessary
bike_rentals_df.drop(['Bike station'], axis=1, inplace=True)

In [28]:
bike_rentals_df.head()

,Bike number,Rental datetime,Return datetime,Rental station,Return station,Duration,Rental station latitude,Rental station longitude,Return station latitude,Return station longitude
0,57719,2019-06-26 00:00:10,2019-06-26 00:06:19,Skarbowców / Wietrzna,Krzycka / Aleja Karkonoska (Park Południowy),00:06:09,51.07329,16.99485,51.074992,17.007058
1,650480,2019-06-26 00:00:13,2019-06-26 00:06:59,Rynek,Plac Legionów,00:06:46,51.109782,17.030175,51.104413,17.022536
2,650988,2019-06-26 00:00:15,2019-06-26 00:13:32,Poza oficjalną stacją,Wałbrzyska - pętla tramwajowa,00:13:17,NaN,NaN,51.06577709842137,16.98857545852661
3,57603,2019-06-26 00:00:21,2019-06-26 00:23:53,Plac Uniwersytecki (UWr),Legnicka / Wejherowska,00:23:32,51.113871,17.034484,51.125276,16.984447
4,650067,2019-06-26 00:00:40,2019-06-26 00:04:40,Powstańców Śląskich (Arkady Wrocławskie),Powstańców Śląskich (Arkady Wrocławskie),00:04:00,51.099713,17.027905,51.099713,17.027905


In [29]:
# Check if we have any missing data:
# - if rental station coordinates are missing -> bikes were rented outside official bike stations
# - if return station coordinates are missing -> bikes were returned outside official bike stations
bike_rentals_df.isna().sum()

Bike number                     0
Rental datetime                 0
Return datetime                 0
Rental station                  0
Return station                  0
Duration                        0
Rental station latitude     55813
Rental station longitude    55813
Return station latitude     53030
Return station longitude    53030
dtype: int64

## Save interim results

In [30]:
# write interim file
bike_rentals_extended_interim_path = os.path.join(data_interim_folder, 'bike_rental_records_extended.csv')
bike_rentals_df.to_csv(path_or_buf=bike_rentals_extended_interim_path, index=False)